# Comparing Speech Analytics Models

In [2]:
import pandas as pd

## Loading csv files

In [3]:
# load the real file
df_real = pd.read_csv('real.csv', names=['file', 'transcript'])
# load the model file
df_model = pd.read_csv('no_ipa.csv', names=['file', 'transcript'])


## Comparing Two lines

In [9]:
# function that neutralizes the string
def neutralize(a_string):
    return a_string.lower()

# function that calculates percentage of similarity
def similarity_score(string_real, string_model, extra_info=False):

    # cut strings to bag of words
    words_real = neutralize(string_real).split(' ')
    count = 0
    error_words = []
    for word in words_real:
        # search word in string_model
        if word in neutralize(string_model):
            count = count + 1
        else:
            error_words.append(word)

    score = count / len(words_real)
    if extra_info:
        return score, error_words

    return score

In [10]:
scr, failed_words = similarity_score(df_real['transcript'][0], df_model['transcript'][0],
                                     extra_info=True)
print(scr)
print(failed_words)

0.8
['alo', 'tardes?', 'hablo', 'perez', 'conversacion', 'socofin', 'credichile', 'inpago', 'credito', 'dosmil', 'veinte,', 'cancelo', 'dia']


## Comparing two dataframes
To compare two dataframes we can use the function we just made but taking care that we
use the correct file from each row.

In [18]:
def similarity_score_dataframes(_df_real, _df_model):
    score_list = []

    for a_file, a_string in zip(_df_model['file'], _df_model['transcript']):
        # first, we search for file in real
        real_str = _df_real[_df_real['file']==a_file]['transcript']

        # files should be 1:1, if not, we riot
        if len(real_str) != 1:
            AttributeError('Dude files are not 1:1 in ' + a_file)

        # otherwise let's continue calculating the score
        print(real_str.array)
        score, fails = similarity_score(real_str.array[0], a_string, extra_info=True)
        score_list.append((a_file, score, fails))

    return pd.DataFrame(score_list, columns=['file', 'score', 'failed_words'])

In [19]:
scr_list = similarity_score_dataframes(df_real, df_model)
scr_list.head()


<PandasArray>
['alo buenas tardes? hablo con don manuel perez rojo si don manuel habla con su ejecutiva virtual para su seguridad esta conversacion podría ser grabada lo estamos llamando de Socofin por encargo de Banco CrediChile para informarle que se encuentra inpago su credito de consumo con fecha cinco de noviembre de dosmil veinte, si usted ya cancelo favor omitir este mensaje que tenga buen dia']
Length: 1, dtype: object
<PandasArray>
['alo buenas tardes? hablo con don manuel perez rojo si don manuel habla con su ejecutiva virtual para su seguridad esta conversacion podría ser grabada lo estamos llamando de Socofin por encargo de Banco CrediChile para informarle que se encuentra inpago su credito de consumo con fecha cinco de noviembre de dosmil veinte, si usted ya cancelo favor omitir este mensaje que tenga buen dia']
Length: 1, dtype: object


,file,score,failed_words
0,100884402_995867770_100179896630201118_2020111...,0.8,"[alo, tardes?, hablo, perez, conversacion, soc..."
1,100884402_995867770_100179896630201118_2020111...,0.8,"[alo, tardes?, hablo, perez, conversacion, soc..."
